In [1]:
!nvidia-smi -L

GPU 0: Tesla V100-PCIE-32GB (UUID: GPU-a9e4cd1d-5e4c-04cc-757a-51d764a9a850)


In [2]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=5H1P3MRWSGYWTI44UOAJ
%env CLEARML_API_SECRET_KEY=tFeY70BbNFX8KLAZZsMDsWmK9vRn24upozXae2Xho08iMkNLfg

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=5H1P3MRWSGYWTI44UOAJ
env: CLEARML_API_SECRET_KEY=tFeY70BbNFX8KLAZZsMDsWmK9vRn24upozXae2Xho08iMkNLfg


In [3]:
# Set to true so ClearML will log the models created by the trainer
%env CLEARML_LOG_MODEL=True

# Set the ClearML task name (default "Trainer")
# %env CLEARML_TASK=

# Set task's project (default "HuggingFace Transformers")
# %env CLEARML_PROJECT=

env: CLEARML_LOG_MODEL=True


In [4]:
%pip install clearml

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
# ClearML installation
%pip install clearml
# Transformers installation
%pip install transformers[torch] datasets
%pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2")


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
     output_dir="matching-vacancies-resume",
     learning_rate=2e-5,
     per_device_train_batch_size=8,
     per_device_eval_batch_size=8,
     num_train_epochs=2,
    save_steps=10000,
)


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
from datasets import load_dataset

dataset = load_dataset("dataset", delimiter='|')


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
# Split the train dataset into train and test sets
train_dataset, test_dataset = dataset['train'].train_test_split(test_size=0.1).values()

In [12]:
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['vacancy'], examples['cv'], truncation=True, padding='max_length', max_length=128)
    # Add labels to the tokenized inputs
    tokenized_inputs['labels'] = examples['response']
    return tokenized_inputs


In [13]:
from datasets import DatasetDict

# Convert to DatasetDict
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

tokenized_datasets = DatasetDict({
    'train': tokenized_train_dataset,
    'test': tokenized_test_dataset
})



Map:   0%|          | 0/302454 [00:00<?, ? examples/s]

Map:   0%|          | 0/33607 [00:00<?, ? examples/s]

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['vacancy', 'cv', 'response', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 302454
    })
    test: Dataset({
        features: ['vacancy', 'cv', 'response', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 33607
    })
})

In [17]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
trainer.train()

/home/jupyter/.local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ClearML Task: created new task id=e0d07cd7baee40e1b46272efa8926b65
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable

Unsupported key of type '<class 'int'>' found when connecting dictionary. It will be converted to str
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.624400
1000,0.574600
1500,0.543500
2000,0.540500
2500,0.532000
3000,0.527900
3500,0.517900
4000,0.506300
4500,0.500700
5000,0.511600


2023-12-18 14:34:30,888 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/HuggingFace%20Transformers/Trainer.e0d07cd7baee40e1b46272efa8926b65/models/checkpoint-10000.zip
2023-12-18 14:38:53,249 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/HuggingFace%20Transformers/Trainer.e0d07cd7baee40e1b46272efa8926b65/models/checkpoint-20000.zip
2023-12-18 14:43:14,354 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/HuggingFace%20Transformers/Trainer.e0d07cd7baee40e1b46272efa8926b65/models/checkpoint-30000.zip
2023-12-18 14:47:54,714 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/HuggingFace%20Transformers/Trainer.e0d07cd7baee40e1b46272efa8926b65/models/checkpoint-40000.zip
2023-12-18 14:52:46,156 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/HuggingFace%20Transformers/Trainer.e0d07cd7baee40e1b46272efa8926b65/models/checkpoint-50000.zip
2023-12-18 14:57:41,070 - clearml.T

KeyboardInterrupt: 

In [20]:
evaluation_results = trainer.evaluate()
print(evaluation_results)


Error in callback <function sys_pipes_forever at 0x7fec10078c10> (for pre_execute):


UnsupportedOperation: fileno

TypeError: intercept_args() got an unexpected keyword argument 'multiprocessing_context'